In [1]:
import pandas as pd
from pyspark.sql import SparkSession

# Criar sessão do Spark com suporte a Hive
spark = SparkSession.builder \
    .appName("PySpark_Escola_DB") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# Criar base de dados
spark.sql("CREATE DATABASE IF NOT EXISTS escola")
spark.sql("USE escola")

# Criar dados para turmas
dados_turmas = pd.DataFrame({
    "id_turma": [1, 2, 3, 4],
    "nome_turma": ["Turma A", "Turma B", "Turma C", "Turma D"],
    "disciplina": ["Matemática", "História", "Física", "Química"],
    "ano": [2024, 2023, 2024, 2023],
    "num_alunos": [30, 25, 20, 28]
})

df_turmas = spark.createDataFrame(dados_turmas)

# Criar tabela permanente para turmas
spark.sql("""
    CREATE TABLE IF NOT EXISTS turmas (
        id_turma INT,
        nome_turma STRING,
        disciplina STRING,
        ano INT,
        num_alunos INT
    ) USING PARQUET
""")

# Inserir dados na tabela
df_turmas.write.mode("overwrite").insertInto("turmas")

# Criar dados para alunos
dados_alunos = pd.DataFrame({
    "id_aluno": [1, 2, 3, 4, 5, 6],
    "nome": ["João Silva", "Maria Santos", "Carlos Nunes", "Ana Costa", "Rui Pereira", "Sara Mendes"],
    "turma": [1, 2, 3, 1, 2, 4],
    "idade": [16, 17, 15, 16, 18, 17],
    "desempenho": ["Bom", "Excelente", "Médio", "Bom", "Fraco", "Excelente"]
})

df_alunos = spark.createDataFrame(dados_alunos)

# Criar tabela permanente para alunos
spark.sql("""
    CREATE TABLE IF NOT EXISTS alunos (
        id_aluno INT,
        nome STRING,
        turma INT,
        idade INT,
        desempenho STRING
    ) USING PARQUET
""")

# Inserir dados na tabela
df_alunos.write.mode("overwrite").insertInto("alunos")

# 🔎 CONSULTAS SQL DIRETAMENTE NA BASE DE DADOS
print("1️⃣ Listar todas as turmas:")
spark.sql("SELECT * FROM turmas").show()

print("2️⃣ Listar todos os alunos:")
spark.sql("SELECT * FROM alunos").show()

# Fechar sessão
spark.stop()


1️⃣ Listar todas as turmas:
+--------+----------+----------+----+----------+
|id_turma|nome_turma|disciplina| ano|num_alunos|
+--------+----------+----------+----+----------+
|       1|   Turma A|Matemática|2024|        30|
|       2|   Turma B|  História|2023|        25|
|       4|   Turma D|   Química|2023|        28|
|       3|   Turma C|    Física|2024|        20|
+--------+----------+----------+----+----------+

2️⃣ Listar todos os alunos:
+--------+------------+-----+-----+----------+
|id_aluno|        nome|turma|idade|desempenho|
+--------+------------+-----+-----+----------+
|       2|Maria Santos|    2|   17| Excelente|
|       6| Sara Mendes|    4|   17| Excelente|
|       3|Carlos Nunes|    3|   15|     Médio|
|       5| Rui Pereira|    2|   18|     Fraco|
|       1|  João Silva|    1|   16|       Bom|
|       4|   Ana Costa|    1|   16|       Bom|
+--------+------------+-----+-----+----------+

